
#  
## Calibrate-Emulate-Sample Demo: Application to a Cloud Microphysics Toy Model

This notebook demonstrates how parameters of cloud droplet size distributions can be learned from data generated by Cloudy, a cloud microphysics toy model that can be downloaded here: https://github.com/climate-machine/Cloudy.jl. 

Cloudy takes the following input:
* parameters defining a **cloud droplet size distribution** (e.g., parameters defining a gamma distribution or an exponential distribution)
* a **''kernel''** defining the efficiency with which cloud droplets collide and coalesce (i.e., stick together and form bigger droplets). 

Cloudy then simulates how the cloud droplet size distribution evolves over time as a result of the droplet interactions defined by the given kernel. It does so by computing a (user-defined) number of moments of the distribution and propagating them forward in time. The simulation stops after a user-defined amount of time $T$. 
Treating the kernel as given, Cloudy can be viewed as a function $G(\theta): \mathbb{R}^p \rightarrow \mathbb{R}^d$ that maps the vector of parameters describing the initial droplet distribution, $\theta = [\theta_1, \theta_2, \ldots, \theta_p]$, to a vector of moments, $M = [M_0^T, M_1^T, \ldots, M_d^T]$

The model equation assumes that we have observations $y$ of the moments coming from some observing system, and that these observations are corrupted by additive noise $\eta$ such that
\begin{equation}
    y = G(\theta) + \eta,
\end{equation}

where the noise $\theta$ is drawn from a d-dimensional Gaussian with distribution $N(0, \Gamma_y)$.

In this demo, we test the calibrate-emulate-sample framework in a **''perfect-model'' setting**, which means that the observations $y$ do not come from some external observing system but are generated by running Cloudy with the  parameters set to their ''true'' values. 

**Given knowledge of the artificial observations $y$, the forward model $G: \mathbb{R}^p \rightarrow \mathbb{R}^d$, and some information about the noise level such as its size or distribution (but not its value), the inverse problem we want to solve is to find the unknown parameters $\theta$.**

A comprehensive treatment of the calibrate-emulate-sample approach to Bayesian inverse problems can be found in Cleary et al., 2020: https://arxiv.org/pdf/2001.03689.pdf

In a one-sentence summary, the **calibrate** step of the algorithm consists of an Ensemble Kalman Inversion that is used to find good training points for a Gaussian process regression, which in turn is used as a replacement (**emulator**) of the original forward model $G$ in the subsequent Markov chain Monte Carlo **sampling** of the posterior distributions of the unknown parameters.

**Final remarks**: Calibrate-emulate-sample was developed to solve inverse problems in situations when running the forward model $G$ is computationally expensive. Cloudy is very cheap to run and it would be computationally feasible to skip the ''calibrate'' and ''emulate'' steps, but this being a demo, we will walk the user through the full pipeline. Applying the full pipeline may also improve the result even when it's not necessary for computational reasons, e.g., because the Gaussian process regression smooths the cost function.

<img src="cloudy_model.jpg" alt="Cloudy" style="width:600px">



##### Load modules

In [1]:
# Import Cloudy modules
using Cloudy.KernelTensors
using Cloudy.PDistributions

In [2]:
# Import modules
using JLD2 # saving and loading Julia arrays
using Distributions  # probability distributions and associated functions 
using StatsBase
using LinearAlgebra
using StatsPlots

In [3]:
# Import Calibrate-Emulate-Sample modules (adjust paths 
# if necessary)
include("CalibrateEmulateSample.jl/src/EKI.jl")
include("CalibrateEmulateSample.jl/src/Truth.jl")
include("CalibrateEmulateSample.jl/src/GPEmulator.jl")
include("CalibrateEmulateSample.jl/src/MCMC.jl")

Main.MCMC

### Set up experiment

In [4]:
# Define the data from which we want to learn 
data_names = ["M0", "M1", "M2"]
moments = [0.0, 1.0, 2.0]
n_moments = length(moments)
param_names = ["N0", "θ", "k"]

FT = Float64

Float64

### Generate truth

In [5]:
# Define the true parameters and distribution as well as the
# priors of the parameters
N0_true = 3650.
θ_true = 4.1
k_true = 2.0
priors = [Distributions.Normal(3000., 500.), # prior on N0
          Distributions.Normal(6.0, 2.0),    # prior on θ
          Distributions.Normal(3.0, 1.5)]    # prior on k 

# We assume that the true particle mass distribution is a 
# Gamma distribution with parameters N0_true, θ_true, k_true
# Note that dist_true has to be a Cloudy distribution, not a 
# "regular" Julia distribution
dist_true = PDistributions.Gamma(N0_true, θ_true, k_true)

# Collision-coalescence kernel to be used in Cloudy
coalescence_coeff = 1/3.14/4
kernel = ConstantCoalescenceTensor(coalescence_coeff)

# Time period over which to run Cloudy
tspan = (0., 0.5)  

# Generate the truth (a Truth.TruthObj)
truth = Truth.run_cloudy_truth(kernel, dist_true, moments, 
                               tspan, data_names, nothing)

Main.Truth.TruthObj(Cloudy.PDistributions.Gamma{Float64}(3650.0, 4.1, 2.0), retcode: Success
Interpolation: 3rd order Hermite
t: [0.0, 1.43246e-6, 2.86492e-6, 6.77977e-6, 1.06946e-5, 1.46095e-5, 1.85243e-5, 2.65758e-5, 4.23722e-5, 7.43235e-5  …  0.407798, 0.415624, 0.42345, 0.431276, 0.443082, 0.454888, 0.466694, 0.4785, 0.490306, 0.5]
u: Array{Float64,1}[[3650.0, 29930.0, 368139.0], [3649.24, 29930.0, 3.68241e5], [3648.48, 29930.0, 3.68343e5], [3646.41, 29930.0, 3.68623e5], [3644.34, 29930.0, 3.68902e5], [3642.27, 29930.0, 369181.0], [3640.2, 29930.0, 3.6946e5], [3635.96, 29930.0, 3.70034e5], [3627.67, 29930.0, 3.71161e5], [3611.0, 29930.0, 3.7344e5]  …  [60.5768, 29930.0, 2.94531e7], [59.4548, 29930.0, 3.00113e7], [58.3736, 29930.0, 3.05695e7], [57.3309, 29930.0, 3.11276e7], [55.8267, 29930.0, 3.19697e7], [54.3994, 29930.0, 3.28117e7], [53.0432, 29930.0, 3.36537e7], [51.753, 29930.0, 3.44958e7], [50.5241, 29930.0, 3.53378e7], [49.5579, 29930.0, 3.60292e7]], [49.5579, 29930.0, 3.60292

# Calibrate: Ensemble Kalman Inversion


In [6]:
log_transform(a::AbstractArray) = log.(a)
exp_transform(a::AbstractArray) = exp.(a)

exp_transform (generic function with 1 method)

In [7]:
N_ens = 50 # number of ensemble members
N_iter = 5 # number of EKI iterations
# initial parameters: N_ens x N_params
initial_params = EKI.construct_initial_ensemble(N_ens, priors; rng_seed=5)
# Note: For the model G (=Cloudy) to run, N0 needs to be 
# nonnegative, and θ and k need to be positive. 
# The EKI update can result in violations of these constraints - 
# therefore, we log-transform the initial ensemble, perform all 
# EKI operations in log space and run G with the exponentiated 
# parameters, which ensures positivity.
ekiobj = EKI.EKIObj(log_transform(initial_params), 
                    param_names, truth.y_t, truth.cov)

Main.EKI.EKIObj(Array{Float64,2}[[8.22474 1.87038 1.11537; 8.04893 1.27945 1.53624; … ; 8.01486 0.57864 1.42383; 8.11053 2.07088 1.49046]], ["N0", "θ", "k"], [49.5579, 29930.0, 3.60292e7], [7.40574 -6.82956 -8.63348; -6.82956 6.53669 8.88536; -8.63348 8.88536 21.557], 50, Array{Float64,2}[], Float64[])

In [8]:
# Initialize a CDistribution with dummy parameters
# The parameters will then be set in run_cloudy_ensemble
dummy = 1.0
dist_type = PDistributions.Gamma(dummy, dummy, dummy)

Cloudy.PDistributions.Gamma{Float64}(1.0, 1.0, 1.0)

In [9]:
# EKI iterations
for i in 1:N_iter
    # Note the exp-transform to ensure positivity of the 
    # parameters
    g_ens = EKI.run_cloudy_ensemble(kernel, dist_type, 
                                    exp_transform(ekiobj.u[end]), 
                                    moments, tspan)
    EKI.update_ensemble!(ekiobj, g_ens) 
end

In [10]:
# EKI results: Has the ensemble collapsed toward the truth?
true_params = PDistributions.get_params(truth.distr_init)
println("True parameters: ")
println(true_params)

println("\nEKI results:")
println(mean(exp_transform(ekiobj.u[end]), dims=1))

True parameters: 
(Symbol[:n, :θ, :k], [3650.0, 4.1, 2.0])

EKI results:
[3489.06 3.63165 2.56231]


#### Plot the evolution of the ensemble

In [11]:
using Plots
gr()

p = plot(exp_transform(ekiobj.u[1][:, 2]), 
         exp_transform(ekiobj.u[1][:, 3]), 
         seriestype=:scatter)
for i in 2:N_iter
    plot!(exp_transform(ekiobj.u[i][:, 2]), 
          exp_transform(ekiobj.u[i][:, 3]), 
          seriestype=:scatter)
end

plot!([θ_true], xaxis="theta", yaxis="k", seriestype="vline", 
      linestyle=:dash, linecolor=:red)
plot!([k_true], seriestype="hline", linestyle=:dash, linecolor=:red)


savefig(p, "EKI_test.png")

# Emulate: Gaussian Process Regression

In [12]:
gppackage = "gp_jl"
yt, yt_cov, yt_covinv, log_u_tp, g_tp = GPEmulator.extract(truth, ekiobj, N_iter)
u_tp = exp_transform(log_u_tp)

250×3 Array{Float64,2}:
 3732.15  6.49075  3.0507  
 3130.44  3.59465  4.64708 
 2875.78  5.89844  2.92272 
 3567.06  4.52769  2.40578 
 2212.71  4.01028  3.96531 
 2550.05  5.44463  2.87062 
 2718.09  5.11     3.74886 
 2486.02  7.95535  1.62365 
 2967.69  4.41036  5.22889 
 3203.76  7.69733  2.23254 
 3706.09  3.11064  0.967374
 3331.03  4.65329  2.39967 
 2844.38  7.57895  5.26408 
    ⋮                      
 3207.32  2.96297  3.14941 
 3234.34  2.92603  3.16273 
 3894.72  4.62297  1.6623  
 3101.99  2.5645   3.76234 
 3548.0   3.89275  2.16701 
 3479.21  3.7053   2.32164 
 3090.54  2.6212   3.69462 
 3981.27  4.7643   1.57794 
 3507.62  3.69074  2.31202 
 3628.6   4.00773  2.05815 
 2870.45  1.94426  5.36293 
 3489.85  3.72149  2.30451 

In [13]:
# Standardize both the data and the parameters
data_mean = vec(mean(g_tp, dims=1))
data_std = vec(std(g_tp, dims=1))
g_tp_zscore = GPEmulator.orig2zscore(g_tp, data_mean, data_std)

param_mean = [mean(prior) for prior in priors]
param_std = [std(prior) for prior in priors]
u_tp_zscore = GPEmulator.orig2zscore(u_tp, param_mean, param_std)

250×3 Array{Float64,2}:
  1.46431     0.245375    0.0338007
  0.260871   -1.20267     1.09805  
 -0.248434   -0.0507798  -0.0515193
  1.13412    -0.736155   -0.396147 
 -1.57459    -0.994859    0.643538 
 -0.899893   -0.277684   -0.0862517
 -0.563828   -0.445       0.499241 
 -1.02797     0.977675   -0.917568 
 -0.0646294  -0.79482     1.48593  
  0.407523    0.848664   -0.51164  
  1.41217    -1.44468    -1.35508  
  0.662051   -0.673357   -0.400219 
 -0.311238    0.789473    1.50939  
  ⋮                                
  0.414639   -1.51852     0.0996054
  0.46867    -1.53698     0.108484 
  1.78943    -0.688515   -0.8918   
  0.203986   -1.71775     0.508223 
  1.09601    -1.05362    -0.555325 
  0.958425   -1.14735    -0.452243 
  0.181087   -1.6894      0.46308  
  1.96253    -0.617849   -0.948042 
  1.01524    -1.15463    -0.458654 
  1.2572     -0.996134   -0.627901 
 -0.259107   -2.02787     1.57529  
  0.979695   -1.13925    -0.463659 

In [14]:
# Fit a Gaussian Process regression to the training points
gpobj = GPEmulator.emulate(u_tp_zscore, g_tp_zscore, gppackage)

Main.GPEmulator.GPObj([1.46431 0.260871 … -0.259107 0.979695; 0.245375 -1.20267 … -2.02787 -1.13925; 0.0338007 1.09805 … 1.57529 -0.463659], [1.00958 0.107934 … -0.396564 0.68338; 2.25359 1.02854 … -0.239267 -0.23928; 1.75654 0.554677 … -0.256791 -0.256822], Any[GP Exact object:
  Dim = 3
  Number of observations = 250
  Mean function:
    Type: MeanZero, Params: Float64[]
  Kernel:
    Type: GaussianProcesses.SumKernel{GaussianProcesses.SumKernel{GaussianProcesses.SEIso{Float64},GaussianProcesses.Mat52Ard{Float64}},GaussianProcesses.Noise{Float64}}
      Type: GaussianProcesses.SumKernel{GaussianProcesses.SEIso{Float64},GaussianProcesses.Mat52Ard{Float64}}
        Type: SEIso{Float64}, Params: [1.46901, 1.04661]        Type: Mat52Ard{Float64}, Params: [12.0182, 9.21207, 10.4125, -16.3933]      Type: Noise{Float64}, Params: [-56.2467]
  Input observations = 
[1.46431 0.260871 … -0.259107 0.979695; 0.245375 -1.20267 … -2.02787 -1.13925; 0.0338007 1.09805 … 1.57529 -0.463659]
  Output ob

In [15]:
# Check how well the Gaussian Process regression predicts on the
# (standardized) true parameters
u_true_zscore = GPEmulator.orig2zscore([N0_true θ_true k_true], 
                                        param_mean, param_std)

y_mean, y_var = GPEmulator.predict(gpobj, reshape(u_true_zscore, 1, :))
y_mean = cat(y_mean..., dims=2)
y_var = cat(y_var..., dims=2)

println("GP prediction on true parameters: ")
println(vec(y_mean) .* data_std + data_mean)
println("true data: ")
println(truth.y_t)

GP prediction on true parameters: 
[49.5579, 29931.0, 3.60292e7]
true data: 
[49.5579, 29930.0, 3.60292e7]


# Sample: Markov chain Monte Carlo

In [16]:
# initial values
u0 = vec(mean(u_tp_zscore, dims=1))
println("initial parameters: ", u0)

# MCMC parameters    
mcmc_alg = "rwm" # random walk Metropolis

initial parameters: [0.313805, -0.644657, -0.322028]


"rwm"

In [17]:
# First let's run a short chain to determine a good step size
burnin = 0
step = 0.1 # first guess
yt_zscore = GPEmulator.orig2zscore(yt, data_mean, data_std)
#yt_cov_zscore = GPEmulator.orig2zscore(yt_cov, data_mean, data_std)

3-element Array{Float64,1}:
  0.903768946851413  
 -0.23926255828549936
 -0.25681263301457585

In [18]:
max_iter = 5000
standardized = true # we are using z scores
mcmc_test = MCMC.MCMCObj(yt_zscore, yt_cov./maximum(yt_cov), 
                        priors, step, u0, 
                        max_iter, mcmc_alg, burnin, standardized)
new_step = MCMC.find_mcmc_step!(mcmc_test, gpobj)

Begin step size search
iteration 0; current parameters [0.313805 -0.644657 -0.322028]
iteration 2000; acceptance rate = 0.46476761619190404, current parameters [0.836495 -1.20015 -0.179453]
new step size: 0.2
iteration 2000; acceptance rate = 0.2383808095952024, current parameters [1.56049 0.0378608 -1.09249]


0.2

In [ ]:
# Now begin the actual MCMC
println("Begin MCMC - with step size ", new_step)

# reset parameters 
burnin = 1000
max_iter = 100000
mcmc = MCMC.MCMCObj(yt_zscore, yt_cov./maximum(yt_cov), priors, 
                    new_step, u0, max_iter, mcmc_alg, burnin,
                    standardized)
MCMC.sample_posterior!(mcmc, gpobj, max_iter)

Begin MCMC - with step size 0.2
iteration 0; current parameters [1.20057 0.497962 -1.37575]
iteration 1000 of 100000; acceptance rate = 0.27472527472527475, current parameters [0.61547 0.372809 -0.342979]
iteration 2000 of 100000; acceptance rate = 0.26936531734132935, current parameters [1.28789 0.768395 -1.5862]
iteration 3000 of 100000; acceptance rate = 0.2642452515828057, current parameters [1.93311 0.175259 -1.04231]
iteration 4000 of 100000; acceptance rate = 0.25818545363659084, current parameters [0.22728 -1.08593 1.14468]
iteration 5000 of 100000; acceptance rate = 0.2563487302539492, current parameters [1.5051 0.622083 -1.21323]
iteration 6000 of 100000; acceptance rate = 0.25462422929511747, current parameters [0.75316 -1.52393 0.30652]
iteration 7000 of 100000; acceptance rate = 0.257820311384088, current parameters [1.59974 -0.708183 -0.703053]
iteration 8000 of 100000; acceptance rate = 0.25821772278465194, current parameters [1.30269 -1.84593 0.119875]
iteration 9000 of

In [ ]:
posterior = MCMC.get_posterior(mcmc)      
post_mean = mean(posterior, dims=1)
post_cov = cov(posterior, dims=1)
println("post_mean")
println(post_mean)
println("post_cov")
println(post_cov)
println("D util")
println(det(inv(post_cov)))
println(" ")

In [ ]:
# Plot the posteriors together with the priors and the true 
# parameter values
using StatsPlots; 

true_values = [N0_true θ_true k_true]
n_params = length(true_values)

for idx in 1:n_params
    if idx == 1
        param = "N0"
        xs = collect(0:1:4000)
    elseif idx == 2
        param = "Theta"
        xs = collect(0:0.01:12.0)
    elseif idx == 3
        param = "k"
        xs = collect(0:0.001:6.0)
    else
        throw("not implemented")
    end

    label = "true " * param
    histogram(posterior[:, idx] .* param_std[idx] .+ param_mean[idx], 
              bins=100, normed=true, fill=:slategray, lab="posterior")
    plot!(xs, mcmc.prior[idx], w=2.6, color=:blue, lab="prior")
    plot!([true_values[idx]], seriestype="vline", w=2.6, lab=label)

    title!(param)
    StatsPlots.savefig("posterior_"*param*".png")
end